In [2]:
%config IPCompleter.greedy=True

# Input The File

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
#Function to ask user for the file location
#and return the dataset
def userFilePath():
    filepath = input('Enter location of the file: ')
    data = pd.read_csv(filepath, low_memory=True)
    return data

# Ask them features and target if model is Supervised

In [26]:
#Input the name of the features and variable
#to put comma after each feature
def selectFeatures():
    x = input("Enter the names of features : ")
    features=x.split(',')
    target = input("Enter the target between '' : ")
    return features, target

# To replace null values

In [37]:
def replaceNull(data, features):
    #to count the number of null values in each column
    count_null=data[data.columns].isna().sum()
    count_null=dict(count_null)
    rows=len(data.index)
    for i in features:
        k=count_null[i]
        if k!=0:
            if k>(rows/2):
                data.drop([i], axis = 1)
            else:
                print('How do u want to replace the null values for feature: ',i)
                print('1. with 0\n2.with mean\n3.with median\n4.with mode')
                ch=int(input())
                if ch==1:
                    data[i] = data[i].fillna(0)
                elif ch==2:
                    mean=data[i].mean()
                    data[i] = data[i].fillna(mean)
                elif ch==3:
                    med=data[i].median()
                    data[i] = data[i].fillna(med)
                elif ch==2:
                    mode=data[i].mode()
                    data[i] = data[i].fillna(mode)
    return data

### Return X and y according to the features entered

In [39]:
#Function to load the features into the X and y respectively and return it
def featget(data, features, target):
    X = data.loc[:,features]
    print(target)
    y = data.loc[:,target]     
    return (X,y)

# Pre-Processing

### OneHotEncoding

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#Function that returns X after performing oneHotEncoding
# columnNumber --> takes the columnNumber for which one hot encoding is to be done
def oneHotEncoding(X, columnNumber):
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [columnNumber])], remainder='passthrough')
    X = np.array(ct.fit_transform(X))
    return X

### LabelEncoder

In [10]:
from sklearn.preprocessing import LabelEncoder

# X--> takes only a single column
def labelEncodingColumn(X):
    le = LabelEncoder()
    X = le.fit_transform(X)
    return X

# Binariztion

In [11]:
from sklearn.preprocessing import Binarizer
def binarize(X_train,X_test):
    binarizer = Binarizer(threshold=0.0).fit(X)
    binary_X = binarizer.transform(X)

# Train-Test Split

In [12]:
#Train-Test split using sklearn
def split_dataset(X, y, testSize, randomState):
    from sklearn.model_selection import train_test_split
    # testSize--> defines the size of the test from the dataset(takes decimal less than 1)
    # randomState --> takes integer input
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSize, random_state = randomState)
    return (X_train, X_test, y_train, y_test)

# Standardization

In [77]:
from sklearn.preprocessing import StandardScaler
def standard(X_train,X_test):
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return (X_train, X_test)

# Normalization

In [14]:
def normalize(X_train,X_test):
    from sklearn.preprocessing import Normalizer
    scaler = Normalizer().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return (X_train, X_test)

# ML Models Start Here

## Linear Regression

In [55]:
def linearReg(X_train,X_test, y_train):
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    return y_pred

## Logistic Regression

In [82]:
def logReg(X_train,X_test, y_train):
    from sklearn.linear_model import LogisticRegression
    logr = LogisticRegression(random_state =0)
    logr.fit(X_train,y_train)
    y_pred = logr.predict(X_test)
    return y_pred

## SV Regression

In [57]:
def svcModel(X_train,X_test, y_train):
    from sklearn.svm import SVC
    svc = SVC(kernel='linear')
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    return y_pred

## KNN

In [66]:
def knnModel(X_train,X_test,y_train,n):
    from sklearn import neighbors
    knn = neighbors.KNeighborsClassifier(n_neighbors=n, metric = 'minkowski', p=2)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    return y_pred

In [67]:
#finding the accuracy of the model
#higher the value of R2 score, the better the model fits your data
def accuracy(y_pred,y_test):
    from sklearn.metrics import r2_score
    return r2_score(y_pred, y_test)

### Calling the functions and predicting

In [83]:
filepath = input('Enter location of the file: ')
df = pd.read_csv(filepath, low_memory=True)
#replacing null values
features, target = selectFeatures()
replaceNull(df, features)

X,y = featget(df,features, target)
    

print("Do you want to perform One Hot Encoding?[y/n]")
if(input() == 'y'):
    X = oneHotEncoding(X, input('Enter the Column number : '))


print('Do you want to perform Label Encoding?[y/n]')
if(input() == 'y'):
    col_num = input('Enter the column name : ')
    X[col_num] = labelEncodingColumn(X[col_num])
    
standardizationYN = input("Do you want to perform Standardization ?[y/n]")
normalizationYN = input("Do you want to perform Normalization ?[y/n]")

#after this we have to call the train test split and other prediction models
print(y_train)
print("Enter 1. for Regression: ")
print("Enter 2. for Classification: ")
c=int(input())
if c == 2:
    n=int(input("Enter the value for K"))
test_size = 0.1
while(test_size <= 0.45):
    X_train,X_test,y_train,y_test = split_dataset(X, y, test_size, 0)
    if(standardizationYN == 'y'):
        X_train, X_test = standard(X_train,X_test)
    
    if(normalizationYN == 'y'):
        X_train, X_test = normalize(X_train,X_test)
    test_size+=0.05
    if c==1:
        y_pred1=linearReg(X_train,X_test, y_train)
        p=accuracy(y_pred1,y_test)
        y_pred2=svcModel(X_train,X_test, y_train)
        q=accuracy(y_pred2,y_test)
        acc={'Linear Regression':p,'Support Vector Regression':q}
        table = pd.DataFrame(acc, columns = ['Linear Regression','Support Vector Regression'], index=['Accuracy'])
        print(table)
        print("Enter 1. for Linear Regression: ")
        print("Enter 2. for Support Vector Regression: ")
        result=int(input())
        if result==1:
            print("Prediction for Linear Regression model: ")
            output = pd.DataFrame({target: y_pred1})
            print(output)
        else:
            print("Prediction for Support Vector Regression model: ")
            output = pd.DataFrame({target: y_pred2})
            print(output)
    elif c==2:  
        y_pred1=logReg(X_train,X_test, y_train)
        p=accuracy(y_pred1,y_test)
        y_pred2=knnModel(X_train,X_test, y_train, n)
        q=accuracy(y_pred2,y_test)
        acc={'Logistic Regression':p,'K-nearest neighbours':q}
        table = pd.DataFrame(acc, columns = ['Logistic Regression','K-nearest neighbours'], index=['Accuracy'])
        print(table)
#         print("Enter 1. for Logistic Regression: ")
#         print("Enter 2. for K-nearest neighbours: ")
#         result=int(input())
#         if result == 1:
#             print("Prediction for Logistic Regression model: ")
#             output = pd.DataFrame({target: y_pred1})
#             print(output)
#         else:
#             print("Prediction for K-nearest neighbours model: ")
#             output = pd.DataFrame({target: y_pred2})
#             print(output)
    #Now when the accuracy calculation function is done 
    #call all the ML model inside them and then print their respective accuracy

Enter location of the file: E:/ML PROJECTS/Arya's SInking titanic/SinkingTitanic/train.csv
Enter the names of features : Pclass,Sex,Age
Enter the target between '' : Survived
How do u want to replace the null values for feature:  Age
1. with 0
2.with mean
3.with median
4.with mode
2
Survived
Do you want to perform One Hot Encoding?[y/n]
n
Do you want to perform Label Encoding?[y/n]
y
Enter the column name : Sex
Do you want to perform Standardization ?[y/n]y
Do you want to perform Normalization ?[y/n]n
375    1
185    0
194    1
401    0
436    0
      ..
835    1
192    1
629    0
559    1
684    0
Name: Survived, Length: 490, dtype: int64
Enter 1. for Regression: 
Enter 2. for Classification: 
2
Enter the value for K7
          Logistic Regression  K-nearest neighbours
Accuracy              0.12037              0.015856
          Logistic Regression  K-nearest neighbours
Accuracy             0.088649              0.109043
          Logistic Regression  K-nearest neighbours
Accuracy   

# What type of ML model?

In [74]:
print(y_train)
print("Enter 1. for Regression: ")
print("Enter 2. for Classification: ")
c=int(input())
if c==1:
    y_pred1=linearReg(X_train,X_test, y_train)
    p=accuracy(y_pred1,y_test)
    y_pred2=svcModel(X_train,X_test, y_train)
    q=accuracy(y_pred2,y_test)
    acc={'Linear Regression':p,'Support Vector Regression':q}
    table = pd.DataFrame(acc, columns = ['Linear Regression','Support Vector Regression'], index=['Accuracy'])
    print(table)
    print("Enter 1. for Linear Regression: ")
    print("Enter 2. for Support Vector Regression: ")
    result=int(input())
    if result==1:
        print("Prediction for Linear Regression model: ")
        output = pd.DataFrame({target: y_pred1})
        print(output)
    else:
        print("Prediction for Support Vector Regression model: ")
        output = pd.DataFrame({target: y_pred2})
        print(output)
elif c==2:  
    y_pred1=logReg(X_train,X_test, y_train)
    p=accuracy(y_pred1,y_test)
    n=int(input("Enter the value for K"))
    y_pred2=knnModel(X_train,X_test, y_train, n)
    q=accuracy(y_pred2,y_test)
    acc={'Logistic Regression':p,'K-nearest neighbours':q}
    table = pd.DataFrame(acc, columns = ['Logistic Regression','K-nearest neighbours'], index=['Accuracy'])
    print(table)
    print("Enter 1. for Logistic Regression: ")
    print("Enter 2. for K-nearest neighbours: ")
    result=int(input())
    if result == 1:
        print("Prediction for Logistic Regression model: ")
        output = pd.DataFrame({target: y_pred1})
        print(output)
    else:
        print("Prediction for K-nearest neighbours model: ")
        output = pd.DataFrame({target: y_pred2})
        print(output)

375    1
185    0
194    1
401    0
436    0
      ..
835    1
192    1
629    0
559    1
684    0
Name: Survived, Length: 490, dtype: int64
Enter 1. for Regression: 
Enter 2. for Classification: 
2
Enter the value for K5
          Logistic Regression  K-nearest neighbours
Accuracy              0.03062              0.034264
Enter 1. for Logistic Regression: 
Enter 2. for K-nearest neighbours: 
1
Prediction for Logistic Regression model: 
     Survived
0           0
1           0
2           0
3           1
4           1
..        ...
396         1
397         0
398         1
399         0
400         0

[401 rows x 1 columns]
